In [1]:
from src.metrics import MetricRegistry

import dotenv
dotenv.load_dotenv()

import os
import sys
import pandas as pd
from sqlalchemy import create_engine
from pyspark.sql import SparkSession

In [2]:
POSTGRES_PASSWORD = os.environ["POSTGRES_PASSWORD"]
POSTGRES_USERNAME = "postgres"
POSTGRES_HOSTNAME = "localhost"
POSTGRES_PORT = "5432"
POSTGRES_URL = f"postgresql+psycopg://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOSTNAME}:{POSTGRES_PORT}/"

In [3]:
metadata_engine = create_engine(POSTGRES_URL + "postgres")
pd.read_sql(
    f"SELECT * FROM metadata.application_3",
    con=metadata_engine,
    index_col="metric_id",
).loc["model_acc"]

OperationalError: (psycopg.OperationalError) connection failed: :1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [2]:
spark = (SparkSession.builder
         .master("local[2]")
         .config(key="spark.sql.caseSensitive", value=True)
         .config(key="spark.sql.execution.arrow.pyspark.fallback.enabled", value=True)
         .config(key="spark.sql.execution.arrow.pyspark.enabled", value=True)
         .config(key="spark.sql.execution.arrow.pyspark.datetime64.enabled", value=True)
         .config(key="spark.jars", value=",".join(["./jar/postgresql-42.6.0.jar",
                                                   "./jar/spark-3.3-bigquery-0.31.1.jar"]))
         .getOrCreate())
spark

23/06/13 19:35:38 WARN Utils: Your hostname, Chuhans-MacBook-Pro-M1.local resolves to a loopback address: 127.0.0.1; using 172.16.46.222 instead (on interface en0)
23/06/13 19:35:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/06/13 19:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.conf.set('parentProject', 'healthy-earth-389717')
spark.conf.set('temporaryGcsBucket', 'verizon-drift-monitoring')

In [5]:
dataset = (spark.read.format('bigquery')
           .options(table='healthy-earth-389717:datasets.baseline_data_table')
           .load())

In [6]:
dataset.show()

+--------------------+--------------------+--------------------+------------+--------------------+-----+--------------------+
|           timestamp|                text|           input_ids|       score|     model_embedding|label|      attention_mask|
+--------------------+--------------------+--------------------+------------+--------------------+-----+--------------------+
|2023-06-06 13:32:...|Read the book, fo...|[101, 3191, 1996,...|0.0105520664|[-0.2706347108, 0...|    0|[1, 1, 1, 1, 1, 1...|
|2023-06-06 13:32:...|Primary plot!Prim...|[101, 3078, 5436,...|0.0056580408|[-0.4939827323, 0...|    0|[1, 1, 1, 1, 1, 1...|
|2023-06-06 13:32:...|What a script, wh...|[101, 2054, 1037,...|0.0069832215|[-0.2732478678, 0...|    0|[1, 1, 1, 1, 1, 1...|
|2023-06-06 13:32:...|I hope this group...|[101, 1045, 3246,...|0.8780788779|[0.0013783015, 0....|    0|[1, 1, 1, 1, 1, 1...|
|2023-06-06 13:32:...|More suspenseful,...|[101, 2062, 23873...|0.9632481337|[-0.3649832308, -...|    0|[1, 1, 1, 1, 1